In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

%pip install -q transformers
%pip install -q loguru
%pip install -q openai langchain langchain[docarray] tiktoken unstructured
%pip install -q llama-index==0.10.4 chromadb loguru pypdf  sentence_transformers llama-index-embeddings-huggingface
%pip install -q pdf2image  pdfminer.six
%pip install -q pillow-heif
%pip install -q unstructured-inference
%pip install -q pikepdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
import openai

from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')

In [3]:
!wget -O Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf https://dl.dropbox.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
FILE_NAME = "./Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf"

--2024-02-19 14:23:18--  https://dl.dropbox.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
Resolving dl.dropbox.com (dl.dropbox.com)... 162.125.5.15, 2620:100:601d:15::a27d:50f
Connecting to dl.dropbox.com (dl.dropbox.com)|162.125.5.15|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876 [following]
--2024-02-19 14:23:18--  https://dl.dropboxusercontent.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.5.15, 2620:100:601d:15::a27d:50f
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.5.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221928 (217K) [applicati

## Similarity K

In [4]:
K = 5

## LangChain

In [5]:
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader

from typing import Any, List
from rich.pretty import pprint

def pretty_print(title: str=None, content: Any=None):
    if title is None:
        print(content)
        return
    print(title)
    pprint(content)

from IPython.display import display, HTML

In [6]:
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [7]:
#@title load data { vertical-output: true}
loader = UnstructuredPDFLoader(FILE_NAME)
docs = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [8]:
#@title splitting { vertical-output: true}
text_splitter = SentenceTransformersTokenTextSplitter()
texts = text_splitter.split_documents(docs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
#@title indexing { vertical-output: true}
emb = OpenAIEmbeddings(openai_api_key=openai.api_key)
vec_db = Chroma.from_documents(documents=texts, embedding=emb)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [10]:
#@title LLM { vertical-output: true}
lc_model = ChatOpenAI(openai_api_key=openai.api_key, temperature=1.5)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
#@title base-retriever { vertical-output: true}
base_retriever = vec_db.as_retriever(search_kwargs={"k":K})

In [12]:
#@title MultiQueryRetriever { vertical-output: true}
final_retriever = MultiQueryRetriever.from_llm(base_retriever, lc_model)

In [13]:
#@title QA chain { vertical-output: true}
tmpl = """
You are an assistant to answer a question from user with a context.

Context:
{context}

Question:
{question}

The response should be presented as a list of key points, after creating the title of the content,
    formatted in HTML with appropriate markup for clarity and organization.
"""
prompt = ChatPromptTemplate.from_template(tmpl)
chain = {"question": RunnablePassthrough(), "context": final_retriever} \
        | prompt \
        | lc_model \
        | StrOutputParser() \

result = chain.invoke("Waht is the doc talking about?")

display(HTML(result))

INFO:langchain.retrievers.multi_query:Generated queries: ['What is the content of the document?', 'Can you provide a summary of the document?', 'What is the main subject being discussed in the document?']


## Llama-Index

In [14]:
TEMPERATURE = 0.0
SIM_TOP_K = 3
RERANK_TOP_K = 3
WIN_SZ = 1024

In [15]:
import asyncio
from tqdm.asyncio import tqdm

import chromadb
from chromadb.api.models.Collection import Collection

from llama_index.core import (
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.core import QueryBundle
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate
from llama_index.core.llms.utils import LLMType
from llama_index.core.schema import NodeWithScore
from llama_index.core.indices.base import BaseIndex
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.embeddings.utils import EmbedType
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.legacy.vector_stores import ChromaVectorStore
from llama_index.core.base.llms.types import CompletionResponse
from llama_index.core.node_parser import SentenceWindowNodeParser

from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.legacy.core.response.schema import RESPONSE_TYPE
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.core.query_engine import CustomQueryEngine, BaseQueryEngine

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
def create_service_context(llm: LLMType, embs: EmbedType) -> ServiceContext:
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=WIN_SZ,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )

    return ServiceContext.from_defaults(
        node_parser=node_parser,
        llm=llm,
        embed_model=embs,
    )

def create_vectors(path: str, collection_name="tmp_collection") -> Collection:
    chroma_client = chromadb.PersistentClient(path)
    # https://github.com/run-llama/llama_index/issues/6528
    return chroma_client.get_or_create_collection(collection_name)

def create_storage_context() -> StorageContext:
    path: str = "./db/LlamaIndexMultiVectorSummaryAgent"
    return StorageContext.from_defaults(
        vector_store=ChromaVectorStore(create_vectors(path=path))
    )


In [17]:
#@title load data { vertical-output: true}
input_files: List[str] = [FILE_NAME]
docs: SimpleDirectoryReader = SimpleDirectoryReader( input_files=input_files).load_data()

In [18]:
#@title LLM { vertical-output: true}
li_model = OpenAI(model="gpt-4-1106-preview", temperature=TEMPERATURE)

In [19]:
#@title splitting and indexing { vertical-output: true}
embs = "local:BAAI/bge-small-en-v1.5"

service_context: ServiceContext = (
    create_service_context(li_model, embs)
)

storage_context: StorageContext = (
    create_storage_context()
)

vector_index: BaseIndex = VectorStoreIndex.from_documents(
    docs,
    service_context=service_context,
    #storage_context=storage_context,
    show_progress=True,
)

<ipython-input-16-0e0cc94f5ebb>:8: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  return ServiceContext.from_defaults(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/160 [00:00<?, ?it/s]

In [20]:
#@title base-retriever { vertical-output: true}
base_retriever = vector_index.as_retriever(similarity_top_k=K)

In [27]:
#@title MultiQueriesRetriever { vertical-output: true}
class MultiQueriesRetriever(BaseRetriever):
    def __init__(self, base_retriever: BaseRetriever, model:OpenAI):
        self.template = PromptTemplate("""You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines only.

    For example, these alternative questions:

    'What is Bill Gates known for?'
│   "Can you provide information about Bill Gates' background?"

    Not:

    '1. What is Bill Gates known for?'
│   "2. Can you provide information about Bill Gates' background?"

    Original question: {question}""")
        self._retrievers = [base_retriever]
        self.base_retriever = base_retriever
        self.model = model

    @classmethod
    def flatten(cls, lst: List[List[Any]]) -> List[Any]:
        return [element for sublist in lst for element in sublist]

    def gen_queries(self, query: str) -> List[str]:
        gen_queries_model = OpenAI(model="gpt-3.5-turbo-0125", temperature=1.5)
        prompt = self.template.format(question=query)
        res = gen_queries_model.complete(prompt)
        return res.text.split("\n")

    async def run_gen_queries(self,generated_queries: List[str]) -> List[NodeWithScore]:
        tasks = list(map(lambda q: self.base_retriever.aretrieve(q), generated_queries))
        res = await tqdm.gather(*tasks)
        return MultiQueriesRetriever.flatten(res)

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        return list()

    async def _aretrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        query: str= query_bundle.query_str
        generated_queries: List[str] = self.gen_queries(query)
        pretty_print("generated_queries", generated_queries)
        node_with_scores = await self.run_gen_queries(generated_queries)
        node_with_scores_uniqued = dict()
        # Simplely removing duplicated nodes in this notebook.
        # For Fusion with ranking, ref:https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
        node_with_scores_uniqued = {node_with_score.get_content(): node_with_score for node_with_score in node_with_scores}
        return node_with_scores_uniqued.values()

mr = MultiQueriesRetriever(base_retriever, li_model)

In [28]:
#@title { vertical-output: true}
ls = mr.gen_queries("Who is Bill Gates?")
pretty_print("ls", ls)
rls = await mr.run_gen_queries(ls)
pretty_print("rls", rls)

ls


[
│   '- "What are Bill Gates\' contributions to the tech industry?"',
│   '- "Can you tell me about Bill Gates\' philanthropic work?"',
│   '- "What positions has Bill Gates held in various organizations?"',
│   '- "How has Bill Gates impacted the world economy?"',
│   '- "What are some notable achievements of Bill Gates?"'
]

100%|██████████| 5/5 [00:00<00:00, 28.58it/s]

rls


[
│   NodeWithScore(
│   │   node=TextNode(
│   │   │   id_='d7b888b6-2791-4394-a834-c73233d4a213',
│   │   │   embedding=None,
│   │   │   metadata={
│   │   │   │   'window': 'U.S.  securities laws in the same manner as if it were a U.S.  domestic \nreporting company.  Regulation FD provides that, when a company \ndiscloses material, non-public information about the company to securities \nmarket professionals or shareholders (where it is reasonably foreseeable \nthat the shareholders will trade on the information), that company must also \ninstantaneously disclose the information to the public.  "Securities market \nprofessionals" generally include analysts, institutional investors and other \ninvestment advisors.  \n \n Only the Investor Relations Manager is authorized to disclose information \nabout the Company in response to requests from securities market \nprofessionals or shareholders.  No Employee shall, without first obtaining \nexplicit approval from the Investor Relations Manager, communicate directly \nwith the investment community (including but not li mited to shareholders, \nanalysts, fund managers or potential investors) any information regarding \nthe Company, regardless whether such information has been disclosed \npublicly or not.  If an Employee receives a request for information from any \nsecurities mark et professionals or shareholders, he or she should promptly \ncontact the Investor Relations Manager to coordinate a response to such \nrequest.  \n o Communications with Media  \nNo Employee shall communicate with any media, publicly or privately, \nwithout first consul ting with the Company\'s personnel responsible for \nreleasing information.  In the event any issue arises that may emerge as a \npotential public relations emergency, such as customer\'s complaints written \non a public forum, Employees shall immediately report such incident to the \npersonnel responsible for releasing information.  If any member of the media \nasks for information about the Company or makes any other requests, \nEmployees shall refer their inquiries or requests to the Company\'s Investor \nRelations Manager .  For more detailed provisions regarding relationships \nwith the media, Employees shall refer to Rules for External \nCommunications by Senior Management and Other Employees to ensure \nthat every Employee will act in accordance with the prescribed conduct \nrules. \n 8.  Business Partners, Suppliers and Clients  \n\uf0a7 8.1 Procurement of Goods and Services We guarantee fair dealings with our \nsuppliers and we choose our suppliers through a fair evaluation of competitive bids. \n No Employee shall discriminate against or deceive a s upplier.  The decision to \nchoose a particular supplier shall be made by reference to the price, service, quality \nand reputation of the supplier as considered in the context of the Company\'s long-\nterm commercial interests.   \n ',
│   │   │   │   'original_text': '"Securities market \nprofessionals" generally include analysts, institutional investors and other \ninvestment advisors.  \n \n',
│   │   │   │   'page_label': '6',
│   │   │   │   'file_name': 'Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf',
│   │   │   │   'file_path': 'Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf',
│   │   │   │   'file_type': 'application/pdf',
│   │   │   │   'file_size': 221928,
│   │   │   │   'creation_date': '2024-02-19',
│   │   │   │   'last_modified_date': '2024-02-19',
│   │   │   │   'last_accessed_date': '2024-02-19'
│   │   │   },
│   │   │   excluded_embed_metadata_keys=[
│   │   │   │   'file_name',
│   │   │   │   'file_type',
│   │   │   │   'file_size',
│   │   │   │   'creation_date',
│   │   │   │   'last_modified_date',
│   │   │   │   'last_accessed_date',
│   │   │   │   'window',
│   │   │   │   'original_text'
│   │   │   ],
│   │   │   excluded_llm_metadata_keys=[
│   │   │   │   'file_name',
│   │   │   │   'file_type',
│   │   │   │   'file_size',
│   │   │   

In [29]:
#@title QA query { vertical-output: true}
query_text = """What is the topic of content?

    The response should be presented as a list of key points, after creating the title of the content,
    formatted in HTML with appropriate markup for clarity and organization.
    """
final_res = await RetrieverQueryEngine(mr).aquery(query_text)
display(HTML(final_res.response))

generated_queries


[
│   'What is the main subject of the content?',
│   'What does the content focus on?',
│   'Can you describe the central theme of the content?',
│   'What is the primary focus of the content?',
│   'What aspect of the content is predominantly discussed?'
]

100%|██████████| 5/5 [00:00<00:00, 31.19it/s]


In [30]:
#@title MultiQueriers { vertical-output: true}
class MultiQueriers:
    def __init__(self,
                 base_retriever: BaseRetriever,
                 base_query_engine: BaseQueryEngine,
                 model:OpenAI,
                 sub_queries_in_bundle_to_answer: bool=True):
        self.base_retriever = base_retriever
        self.base_query_engine = base_query_engine
        self.model = model
        self.sub_queries_in_bundle_to_answer = sub_queries_in_bundle_to_answer
        self.gen_q_template = PromptTemplate("""You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines only.

    For example, these alternative questions:

    'What is Bill Gates known for?'
│   "Can you provide information about Bill Gates' background?"

    Not:

    '1. What is Bill Gates known for?'
│   "2. Can you provide information about Bill Gates' background?"

    Original question: {question}""")
        self.qa_prompt_template = PromptTemplate("""Here is the question you need to answer:

    \n --- \n {query_str} \n --- \n

    Here is any available background question + answer pairs:

    \n --- \n {q_a_pairs} \n --- \n

    Here is additional context relevant to the question:

    \n --- \n {context_str} \n --- \n

    Use the above context and any background question + answer pairs to answer the question: \n {query_str}
    """)

    def gen_queries(self, query: str) -> List[str]:
        gen_queries_model = OpenAI(model="gpt-3.5-turbo-0125", temperature=1.5)
        prompt = self.gen_q_template.format(question=query)
        res = gen_queries_model.complete(prompt)
        return res.text.split("\n")

    def query_by_retriever(self, query_str: str) ->str:
        nodes = self.base_retriever.retrieve(query_str)
        res = "\n\n".join([n.node.get_content() for n in nodes])
        return res

    async def run_gen_queries(self,generated_queries: List[str]) -> str:
        sub_query_qa_pairs = list()
        if self.sub_queries_in_bundle_to_answer:
            # Answer all queries in one bundle.
            tasks = list(map(lambda q: self.base_query_engine.aquery(q), generated_queries))
            res = await tqdm.gather(*tasks)
            for idx, (query, answer) in enumerate(zip(generated_queries, res)):
                qa_pair = f"Question {idx}: {query}\nAnswer: {answer}\n"
                sub_query_qa_pairs.append(qa_pair)
            pretty_print("sub_query_qa_pairs", sub_query_qa_pairs)
            return "\n\n".join(sub_query_qa_pairs)
        else:
            # Answer queries in step down.
            # One sub-query will be answer based on the context of sub-query,
            # history of pairs of previous queries and answers.
            for idx, query in enumerate(generated_queries):
                pretty_print(f"{idx}. query", query)
                pretty_print(f"{idx}. sub_query_qa_pairs", sub_query_qa_pairs)
                context_str = self.query_by_retriever(query)
                sub_query = self.qa_prompt_template.format(
                                  query_str=query,
                                  q_a_pairs="\n\n".join(sub_query_qa_pairs),
                                  context_str=context_str
                )
                pretty_print("sub_query", sub_query)
                answer: str = self.model.complete(sub_query)
                qa_pair = f"Question {idx}: {query}\nAnswer: {answer}\n"
                sub_query_qa_pairs.append(qa_pair)
            return "\n\n".join(sub_query_qa_pairs)

    def query(self, query_str: str) -> CompletionResponse:
        return ""

    async def aquery(self, query_str: str) -> CompletionResponse:
        generated_queries: List[str] = self.gen_queries(query_str)
        sub_query_qa_pairs: str = await self.run_gen_queries(generated_queries)
        context_str = self.query_by_retriever(query_str)
        final_query: str = self.qa_prompt_template.format(query_str=query_str,
                                q_a_pairs=sub_query_qa_pairs,
                                context_str=context_str)
        pretty_print("final_query", final_query)
        response: str = self.model.complete(final_query)
        return response


In [33]:
#@title QA query, sub-queries in bundle to answer{ vertical-output: true}
query_text = """What is the topic of content?

    The response should be presented as a list of key points, after creating the title of the content,
    formatted in HTML with appropriate markup for clarity and organization.
    """
mqe=MultiQueriers(base_retriever,  vector_index.as_query_engine(), li_model, sub_queries_in_bundle_to_answer=True)
final_res = await mqe.aquery(query_text)
display(HTML(final_res.text))


100%|██████████| 5/5 [00:07<00:00,  1.55s/it]

sub_query_qa_pairs


[
│   "Question 0: What is the main theme of the content?\nAnswer: The main theme of the content is the importance of employees to the company's success and a summary of the company's employment practices.\n",
│   "Question 1: What does the content primarily focus on?\nAnswer: The content primarily focuses on the principles and values that guide the conduct and ethics of a company. It emphasizes the importance of employees as key assets to the company's success and acknowledges the impact of media and public perception on the company's reputation.\n",
│   "Question 2: What is the central subject matter of the content?\nAnswer: The central subject matter of the content is the company's employment practices, specifically highlighting the value placed on employees as important assets and keys to the company's success.\n",
│   "Question 3: What is the subject addressed in the content?\nAnswer: The subject addressed in the content pertains to the protocol for employees of a company in responding to information requests from securities market professionals or shareholders, and a summary of the company's employment practices.\n",
│   'Question 4: What is the overarching topic of the content?\nAnswer: The overarching topic of the content is the guidelines for ethical business conduct and practices, specifically highlighting the importance of scrutinizing financial inconsistencies, questionable transactions, and attempts to bypass standard review and approval processes. Additionally, it acknowledges that not all possible scenarios can be anticipated or covered by the guidelines.\n'
]

final_query


"Here is the question you need to answer:\n\n    \n --- \n What is the topic of content?\n\n    The response should be presented as a list of key points, after creating the title of the content,\n    formatted in HTML with appropriate markup for clarity and organization.\n     \n --- \n\n\n    Here is any available background question + answer pairs:\n\n    \n --- \n Question 0: What is the main theme of the content?\nAnswer: The main theme of the content is the importance of employees to the company's success and a summary of the company's employment practices.\n\n\nQuestion 1: What does the content primarily focus on?\nAnswer: The content primarily focuses on the principles and values that guide the conduct and ethics of a company. It emphasizes the importance of employees as key assets to the company's success and acknowledges the impact of media and public perception on the company's reputation.\n\n\nQuestion 2: What is the central subject matter of the content?\nAnswer: The central subject matter of the content is the company's employment practices, specifically highlighting the value placed on employees as important assets and keys to the company's success.\n\n\nQuestion 3: What is the subject addressed in the content?\nAnswer: The subject addressed in the content pertains to the protocol for employees of a company in responding to information requests from securities market professionals or shareholders, and a summary of the company's employment practices.\n\n\nQuestion 4: What is the overarching topic of the content?\nAnswer: The overarching topic of the content is the guidelines for ethical business conduct and practices, specifically highlighting the importance of scrutinizing financial inconsistencies, questionable transactions, and attempts to bypass standard review and approval processes. Additionally, it acknowledges that not all possible scenarios can be anticipated or covered by the guidelines.\n \n --- \n\n\n    Here is additional context relevant to the question:\n\n    \n --- \n If an Employee receives a request for information from any \nsecurities mark et professionals or shareholders, he or she should promptly \ncontact the Investor Relations Manager to coordinate a response to such \nrequest.  \n\n\no Valuing Employees a s Important Assets  \nThe Company believes its Employees are important assets and keys to its \nsuccess. \n\nThe following is intended to be a summary of the Company's employment practices. \n\n\nParticular attention should be paid to:  \no financial results that seem inconsistent with the performance of the \nunderlying business;  \no transactions that do not seem to have an obvious business purpose; and  \no requests to circumvent ordinary review and approval procedures.  \n\n\nWhat is written or \nsaid about the Company in the news media and investment community directly \nimpacts its reputation, positively or negatively.  \n --- \n\n\n    Use the above context and any background question + answer pairs to answer the question: \n What is the topic of content?\n\n    The response should be presented as a list of key points, after creating the title of the content,\n    formatted in HTML with appropriate markup for clarity and organization.\n    \n    "

In [32]:
#@title QA query, sub-queries step down to generate final answer { vertical-output: true}
query_text = """What is the topic of content?

    The response should be presented as a list of key points, after creating the title of the content,
    formatted in HTML with appropriate markup for clarity and organization.
    """
mqe=MultiQueriers(base_retriever,  vector_index.as_query_engine(), li_model, sub_queries_in_bundle_to_answer=False)
final_res = await mqe.aquery(query_text)
display(HTML(final_res.text))

0. query


'What is the main subject of the content?'

0. sub_query_qa_pairs


[]

sub_query


...

[
│   "Question 0: What is the main subject of the content?\nAnswer: The main subject of the content is the Company's employment practices, specifically emphasizing the value of employees as important assets and key contributors to the Company's success. It also touches on the broader scope of the Company's assets and the impact of public perception on its reputation.\n",
│   "Question 1: What does the content primarily focus on?\nAnswer: The content primarily focuses on the Company's employment practices, highlighting the importance of employees as key assets and their significant role in the success of the Company. It also addresses the impact of public perception on the Company's reputation, as influenced by media and investment community discussions. Additionally, the content includes a cautionary note on financial results, business transactions, and adherence to review and approval procedures, which are likely related to maintaining the integrity and reputation of the Company. It also briefly mentions other assets of the Company such as information, technical materials, software, information systems, construction, equipment, files, and cash.\n",
│   "Question 2: What subject matter is covered in the content?\nAnswer: The subject matter covered in the content includes the Company's employment practices, with a particular emphasis on the value of employees as important assets and their crucial role in the success of the Company. It also covers the range of the Company's assets, which encompasses information, technical materials, software, information systems, construction, equipment, files, and cash. Additionally, the content touches on the Company's public communications, including those made to the Securities and Exchange Commission (SEC), and the importance of public perception on the Company's reputation. The content further mentions the Company's internal controls, such as written policies and procedures, management reviews and monitoring, budgetary control, and various examinations and settlements, which are part of the efforts to maintain the integrity and reputation of the Company.\n",
│   "Question 3: Can you summarize the main topic discussed in the content?\nAnswer: The main topic discussed in the content is the Company's employment practices, emphasizing the importance of employees as vital assets and their significant role in contributing to the Company's success. The content also addresses the impact of the Company's public reputation as influenced by media and the investment community, and the need for fair and accurate communication regarding competitors based on publicly available information. Additionally, it mentions the role of securities market professionals, such as analysts and institutional investors, in relation to the Company.\n"
]

sub_query


"Here is the question you need to answer:\n\n    \n --- \n What is the central theme of the content? \n --- \n\n\n    Here is any available background question + answer pairs:\n\n    \n --- \n Question 0: What is the main subject of the content?\nAnswer: The main subject of the content is the Company's employment practices, specifically emphasizing the value of employees as important assets and key contributors to the Company's success. It also touches on the broader scope of the Company's assets and the impact of public perception on its reputation.\n\n\nQuestion 1: What does the content primarily focus on?\nAnswer: The content primarily focuses on the Company's employment practices, highlighting the importance of employees as key assets and their significant role in the success of the Company. It also addresses the impact of public perception on the Company's reputation, as influenced by media and investment community discussions. Additionally, the content includes a cautionary note on financial results, business transactions, and adherence to review and approval procedures, which are likely related to maintaining the integrity and reputation of the Company. It also briefly mentions other assets of the Company such as information, technical materials, software, information systems, construction, equipment, files, and cash.\n\n\nQuestion 2: What subject matter is covered in the content?\nAnswer: The subject matter covered in the content includes the Company's employment practices, with a particular emphasis on the value of employees as important assets and their crucial role in the success of the Company. It also covers the range of the Company's assets, which encompasses information, technical materials, software, information systems, construction, equipment, files, and cash. Additionally, the content touches on the Company's public communications, including those made to the Securities and Exchange Commission (SEC), and the importance of public perception on the Company's reputation. The content further mentions the Company's internal controls, such as written policies and procedures, management reviews and monitoring, budgetary control, and various examinations and settlements, which are part of the efforts to maintain the integrity and reputation of the Company.\n\n\nQuestion 3: Can you summarize the main topic discussed in the content?\nAnswer: The main topic discussed in the content is the Company's employment practices, emphasizing the importance of employees as vital assets and their significant role in contributing to the Company's success. The content also addresses the impact of the Company's public reputation as influenced by media and the investment community, and the need for fair and accurate communication regarding competitors based on publicly available information. Additionally, it mentions the role of securities market professionals, such as analysts and institutional investors, in relation to the Company.\n \n --- \n\n\n    Here is additional context relevant to the question:\n\n    \n --- \n o Valuing Employees a s Important Assets  \nThe Company believes its Employees are important assets and keys to its \nsuccess. \n\nThe following is intended to be a summary of the Company's employment practices. \n\n\nParticular attention should be paid to:  \no financial results that seem inconsistent with the performance of the \nunderlying business;  \no transactions that do not seem to have an obvious business purpose; and  \no requests to circumvent ordinary review and approval procedures.  \n\n\nThe \nCompany's assets include, without limitation, information, technical materials, \nsoftware, information systems, construction, equipment, files and cash. \n\nWhat is written or \nsaid about the Company in the news media and investment community directly \nimpacts its reputation, positively or negatively.  \n --- \n\n\n    Use the above context and any background question + answer pairs to answer the question: \n Wh

final_query


"Here is the question you need to answer:\n\n    \n --- \n What is the topic of content?\n\n    The response should be presented as a list of key points, after creating the title of the content,\n    formatted in HTML with appropriate markup for clarity and organization.\n     \n --- \n\n\n    Here is any available background question + answer pairs:\n\n    \n --- \n Question 0: What is the main subject of the content?\nAnswer: The main subject of the content is the Company's employment practices, specifically emphasizing the value of employees as important assets and key contributors to the Company's success. It also touches on the broader scope of the Company's assets and the impact of public perception on its reputation.\n\n\nQuestion 1: What does the content primarily focus on?\nAnswer: The content primarily focuses on the Company's employment practices, highlighting the importance of employees as key assets and their significant role in the success of the Company. It also addresses the impact of public perception on the Company's reputation, as influenced by media and investment community discussions. Additionally, the content includes a cautionary note on financial results, business transactions, and adherence to review and approval procedures, which are likely related to maintaining the integrity and reputation of the Company. It also briefly mentions other assets of the Company such as information, technical materials, software, information systems, construction, equipment, files, and cash.\n\n\nQuestion 2: What subject matter is covered in the content?\nAnswer: The subject matter covered in the content includes the Company's employment practices, with a particular emphasis on the value of employees as important assets and their crucial role in the success of the Company. It also covers the range of the Company's assets, which encompasses information, technical materials, software, information systems, construction, equipment, files, and cash. Additionally, the content touches on the Company's public communications, including those made to the Securities and Exchange Commission (SEC), and the importance of public perception on the Company's reputation. The content further mentions the Company's internal controls, such as written policies and procedures, management reviews and monitoring, budgetary control, and various examinations and settlements, which are part of the efforts to maintain the integrity and reputation of the Company.\n\n\nQuestion 3: Can you summarize the main topic discussed in the content?\nAnswer: The main topic discussed in the content is the Company's employment practices, emphasizing the importance of employees as vital assets and their significant role in contributing to the Company's success. The content also addresses the impact of the Company's public reputation as influenced by media and the investment community, and the need for fair and accurate communication regarding competitors based on publicly available information. Additionally, it mentions the role of securities market professionals, such as analysts and institutional investors, in relation to the Company.\n\n\nQuestion 4: What is the central theme of the content?\nAnswer: The central theme of the content is the emphasis on the Company's employment practices and the recognition of employees as critical assets essential to the Company's success. It underscores the importance of maintaining integrity in financial reporting, the purposefulness of business transactions, and adherence to established review and approval procedures. Additionally, the content highlights the significance of the Company's reputation as influenced by public perception, shaped by discussions in the news media and the investment community.\n \n --- \n\n\n    Here is additional context relevant to the question:\n\n    \n --- \n If an Employee receives a request for information from any \nsecurities mark et professionals or shareholders, he or she should promptly \